In [93]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error, recall_score, precision_score, confusion_matrix, f1_score
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier


data = pd.read_csv('contest_train.csv')
data = data.fillna(data.mean())
data = data.drop(('ID'), axis = 1)

# Новые объекты
test = pd.read_csv('contest_test.csv')
test = test.fillna(test.mean())
ids = test['ID']
test = test.drop(('ID'), axis=1)




In [94]:
from sklearn.cross_validation import train_test_split

X = data.drop(('TARGET'), axis=1)
y = data['TARGET'][:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)
X_train = pd.DataFrame(X_train, columns=list(X_train))
X_test = pd.DataFrame(X_test, columns=list(X_test))
N_train = X_train.shape
N_test = X_test.shape
N_train, N_test


((17164, 260), (7357, 260))

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = RandomForestClassifier(random_state=42, n_jobs=-1)
result = cross_val_score(model, X_train, y_train, cv=skf)

print("CV accuracy score: {:.2f}%".format(result.mean()*100))


CV accuracy score: 70.07%


In [95]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=45, max_depth = 24, min_samples_leaf=1)  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('macro_f1_score: ', f1_score(y_test, y_pred, average='macro'))
print('accuracy: ', accuracy_score(y_test, y_pred))

confusion_matrix(y_test, y_pred)


macro_f1_score:  0.389133860006
accuracy:  0.716868288705


array([[5134,   60,   35],
       [1567,   56,   39],
       [ 346,   36,   84]], dtype=int64)

In [92]:
# Запись для предсказания новых объектов

model = RandomForestClassifier(n_estimators=35, max_depth = 24, min_samples_leaf=2)
model.fit(X, y)
y_pred = model.predict(test)


0.71890716324588821

In [88]:
# Запись в файл 

export_data = pd.DataFrame(columns=['ID', 'TARGET'])
export_data['ID'] = ids
export_data['TARGET'] = y_pred
export_data.to_csv('contest_answer.csv')        

In [89]:
# Второе задание 

zero = []
first = []
ids_first = []
second = []
ids_sec = []

for i in range(len(test)):
    if y_pred[i] == np.int64(0):
        zero.append(y_pred[i])
    if y_pred[i] == np.int64(1):
        first.append(y_pred[i])
        ids_first.append(ids[i])
    if y_pred[i] == np.int64(2):
        second.append(y_pred[i])
        ids_sec.append(ids[i])
        
len(zero), len(first), len(second)

(8732, 380, 372)

In [90]:
res = pd.DataFrame(columns=['ID', 'TARGET'])
res['ID'] = ids_first
res['TARGET'] = first
res = res.sample(n=366)

res1 = pd.DataFrame(columns=['ID', 'TARGET'])
res1['ID'] = ids_sec
res1['TARGET'] = second
res1 = res1.sample(n=200)
result = res.append(res1)


In [91]:
result.to_csv('contest_segments.csv')